# 1. Thư viện

In [1]:
import os
import json
import re
from docx import Document

# 2. Thực thi

## 2.1. Hàm đếm từ

In [2]:
def count_tokens_simple(text):
    # Tách từ bằng cách sử dụng khoảng trắng
    tokens = text.split()
    return len(tokens)

## 2.2. Hàm xử lý tên file

In [3]:
def process_file_name(file_name):
    parts = file_name.replace('.docx', '').split('_')
    
    if len(parts) != 7:
        return None  # Nếu file không hợp lệ
    
    loai_van_ban = parts[0]
    noi_ban_hanh = parts[1]
    so_hieu = parts[2].replace("-", "/")
    linhvuc_nganh = parts[3]
    ngay_ban_hanh = parts[4].replace("-", "/")
    ngay_hieu_luc = parts[5] if parts[5] == "Đã biết" else parts[5].replace("-", "/")
    chu_de = parts[6]
    
    return {
        "loai_van_ban": loai_van_ban,
        "noi_ban_hanh": noi_ban_hanh,
        "so_hieu": so_hieu,
        "linhvuc_nganh": linhvuc_nganh,
        "ngay_ban_hanh": ngay_ban_hanh,
        "ngay_hieu_luc": ngay_hieu_luc,
        "chu_de": chu_de
    }

## 2.3. Hàm trích xuất các từ khóa

In [4]:
def extract_keywords(text):
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

## 2.4. Hàm đọc và xử lý nội dung của file .docx

In [5]:
def process_docx(file_path, max_tokens):
    document = Document(file_path)
    
    current_chapter = None
    current_section = None
    current_mini_section = None
    data = []
    
    for i in range(len(document.paragraphs)):
        paragraph_text = document.paragraphs[i].text.strip()
        
        if paragraph_text.startswith("Chương"):
            current_chapter = paragraph_text
            current_section = None
            current_mini_section = None
            if i + 1 < len(document.paragraphs):
                next_paragraph = document.paragraphs[i + 1].text.strip()
                if not next_paragraph.startswith(("Mục", "Tiểu mục", "Điều")):
                    current_chapter += f": {next_paragraph}"
            continue
        
        if paragraph_text.startswith("Mục"):
            current_section = paragraph_text
            current_mini_section = None
            continue
        
        if paragraph_text.startswith("Tiểu mục"):
            current_mini_section = paragraph_text
            continue
        
        if paragraph_text.startswith("Điều"):
            article = paragraph_text
            content = []
            
            for j in range(i + 1, len(document.paragraphs)):
                next_paragraph = document.paragraphs[j].text.strip()
                if next_paragraph.startswith(("Chương", "Mục", "Tiểu mục", "Điều")):
                    break
                content.append(next_paragraph)
            
            # Chuyển đổi nội dung thành chuỗi với dấu xuống dòng
            content_text = "\n".join(content)
            combine_article_content = f"{article}\n{content_text}"
            key_words = extract_keywords(combine_article_content)

            # Kiểm tra số lượng tokens của nội dung
            if count_tokens_simple(content_text) > max_tokens - count_tokens_simple(article):
                # Tách nội dung thành từng phần dựa trên số thứ tự (1., 2., 3., ...)
                numbered_sections = re.split(r'(?=\d+\.\s)', content_text)

                current_content = ""
                for section in numbered_sections:
                    # Kiểm tra số lượng tokens sau khi cộng thêm section
                    if count_tokens_simple(current_content + section) <= max_tokens - count_tokens_simple(article):
                        current_content += section + "\n"  # Thêm xuống dòng
                    else:
                        # Lưu phần nội dung hiện tại
                        data.append({
                            "Chapter": current_chapter,
                            "Section": current_section if current_section else None,
                            "Mini-Section": current_mini_section if current_mini_section else None,
                            "Article": article,
                            "Content": current_content.strip(),
                            "combine_Article_Content": f"{article}\n{current_content.strip()}",
                            "key_words": key_words
                        })
                        current_content = section + "\n"  # Thêm xuống dòng

                # Lưu phần nội dung còn lại nếu có
                if current_content:
                    data.append({
                        "Chapter": current_chapter,
                        "Section": current_section if current_section else None,
                        "Mini-Section": current_mini_section if current_mini_section else None,
                        "Article": article,
                        "Content": current_content.strip(),
                        "combine_Article_Content": f"{article}\n{current_content.strip()}",
                        "key_words": key_words
                    })
            else:
                # Nếu nội dung nhỏ hơn max_tokens - length of article
                data.append({
                    "Chapter": current_chapter,
                    "Section": current_section if current_section else None,
                    "Mini-Section": current_mini_section if current_mini_section else None,
                    "Article": article,
                    "Content": content_text.strip(),
                    "combine_Article_Content": f"{article}\n{content_text.strip()}",
                    "key_words": key_words
                })

    return data

## 2.5. Hàm chính xử lý tất cả các file trong thư mục

In [6]:
def process_folder(folder_path, output_json_path, max_tokens):
    all_data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".docx"):
            file_metadata = process_file_name(file_name)
            if file_metadata is None:
                continue
            
            file_path = os.path.join(folder_path, file_name)
            docx_data = process_docx(file_path, max_tokens=max_tokens)
            
            for entry in docx_data:
                combined_entry = {**file_metadata, **entry}
                all_data.append(combined_entry)
    
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"Dữ liệu đã được lưu vào file {output_json_path}")

## 2.6. Thực thi

In [7]:
# Đường dẫn tới thư mục và file đầu ra
folder_path = r'data\trich_dan_luat\docx\Luat'
json_file_path = r'data\trich_dan_luat\json\test_Combined_Output_Token.json'
max_tokens = 2048  # Giới hạn số token

In [8]:
# Chạy chương trình chính
process_folder(folder_path, json_file_path, max_tokens)

Dữ liệu đã được lưu vào file data\trich_dan_luat\json\test_Combined_Output_Token.json
